<h2 align="center">Парсинг музыки Yandex.Music</h2> 

Загрузим все песни по 10 жанрам в единый словарь с ключом - исполнителем и значением - песней

### 1.Загружаем библиотеки

In [91]:
import requests
from bs4 import BeautifulSoup

### 2. Загружаем URL плейлистов по нужным жанрам

In [43]:
urls_yandex = {
    'blues':'https://music.yandex.ru/users/ya.musicgenre/playlists/1069',
    'classical':'https://music.yandex.ru/users/yamusic-top/playlists/1138',
    'country':'https://music.yandex.ru/users/ya.musicgenre/playlists/1040',
    'disco':'https://music.yandex.ru/users/music.partners/playlists/2061',
    'hip hop':'https://music.yandex.ru/users/yamusic-top/playlists/1039',
    'jazz':'https://music.yandex.ru/users/ya.musicgenre/playlists/1125',
    'metal':'https://music.yandex.ru/users/yamusic-top/playlists/1080',
    'pop':'https://music.yandex.ru/users/yamusic-top/playlists/1010',
    'reggae':'https://music.yandex.ru/users/yamusic-top/playlists/1034',
    'rock':'https://music.yandex.ru/users/yamusic-top/playlists/1013'
}
songs_yandex = dict(zip(urls_yandex.keys(),[{} for x in range(10)]))

### 3. Парсим плейлисты

In [76]:
from tqdm import tqdm
for genre,url in tqdm(zip(urls_yandex.keys(),urls_yandex.values())):
    response = requests.get(url)
    tree = BeautifulSoup(response.text, features="html.parser")
    song_names = tree.find_all('div', {'class': 'd-track__name'})
    song_artists = tree.find_all('div', {'class': 'd-track__meta'})
    song_names = BeautifulSoup(str(song_names))
    songs = dict()
    info_names = song_names.find_all('div')
    info_artists = song_artists
    for name,artist in zip(info_names,info_artists):
        songs[name['title']] = artist.find('a')['title']
    songs_yandex[genre] = songs

10it [00:09,  1.06it/s]


### 4. Загружаем в JSON

In [90]:
import json
with open('songs_yandex.json','w') as f:
    json.dump(songs_yandex,f)

---

<h2 align="center">Загрузка музыки songs_yandex</h2> 

Следующее, что нужно, это выгрузить все песни из словаря __songsyandex__ \
Сделаем это через библиотеку yandex-music

In [238]:
from yandex_music import Client
import os
import shutil


In [96]:
token = 'AQAAAABXD1fqAAG8XuQ5vS3CT0pSrrMhtzl_sPA'

In [100]:
client = Client(token).init()

In [297]:
c=0
for genre in songs_yandex:
    os.mkdir(genre)
    for artist, song in (zip(songs_yandex[genre].values(), songs_yandex[genre].keys())):
        searcher = client.search(artist+' '+song).best.result
        if genre != 'classical':
            searcher.download(os.path.abspath(genre)+'/'+searcher.title+'.wav')
        else:
            c+=1
            searcher.download(os.path.abspath(genre)+'/'+str(c)+'.wav')
    print(genre,' - COMPELETED')

In [ ]:
for genre in ['pop','reggae','rock']:
    os.mkdir(genre)
    for artist, song in (zip(songs_yandex[genre].values(), songs_yandex[genre].keys())):
        searcher = client.search(artist+' '+song).best.result
        try:
            searcher.download(os.path.abspath(genre)+'/'+searcher.title+'.wav')
        except:
            pass
    print('COMPLETED')

In [ ]:
os.mkdir('rock')
for artist, song in (zip(songs_yandex[genre].values(), songs_yandex[genre].keys())):
    searcher = client.search(artist+' '+song).best.result
    try:
        searcher.download(os.path.abspath(genre)+'/'+searcher.title+'.wav')
    except:
        pass
print('COMPLETED')

#### Как все происходило?

Переменная __token__ - это токен аккаунта Yandex.Music, который нужно получить для работы с библиотекой. После инициализации Client мы можем выполнять множество действий, но займемся одним - поиском нужных песен через __.search__ \
После того как трек найден, берем лучший результат и скачиваем его в формате __.wav__